In [3]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [14]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

# Iterate through all pages
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain article information
    articles = soup.find_all('li', class_='slide')

    # Iterate through each article
    for article in articles:
        news_title = article.find('div', class_="content_title").text
        news_paragraph = article.find('div', class_="article_teaser_body").text
        link = article.a['href']
        news_article_link = 'https://mars.nasa.gov/' + link
        print('-----------')
        print(news_title)
        print(news_paragraph)
        print(news_article_link)

    # Click the 'Next' button on each page
    try:
        browser.click_link_by_partial_text('next')
          
    except:
        print("Scraping Complete")




-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

Scraping Complete
-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowe

-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

...

Scraping Complete
-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowe

Scraping Complete
-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowe

-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

Scraping Complete
-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowe

-----------
NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/
-----------
New Finds for Mars Rover, Seven Years After Landing
NASA's Curiosity rover is discovering odd rocks halfway into a region called the "clay-bearing unit."
https://mars.nasa.gov//news/8498/new-finds-for-mars-rover-seven-years-after-landing/
-----------
MEDLI2 Installation on Mars 2020 Aeroshell Begins
Hardware installed onto NASA's Mars 2020 entry vehicle this week will help to increase the safety of future Mars landings.
https://mars.nasa.gov//news/8497/medli2-installation-on-mars-2020-aeroshell-begins/
-----------
NASA's Mars 2020 Rover Does Biceps Curls 
In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.
h

Visit the url for JPL Featured Space Image here.

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [28]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Navigate to page with image info
browser.click_link_by_partial_text('FULL IMAGE')
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

# Scrape image info
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image = soup.find('figure', class_='lede')
link = image.a['href']
featured_image_url = 'https://www.jpl.nasa.gov' + link
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17440_hires.jpg


Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

[]


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string.

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.